# NFA and DFA algorithms

Here is a dfa with 2 states, 0 and 1. where 0 is both a start and a final state

In [9]:
dfa1 = {'start':0,'final':{0},'edges':{(0,1,'a'),(1,0,'b'),(1,0,'c')}}


# Algorithm to check if a DFA accepts a string

In [10]:
def edge(dfa,state,c):
    next_states = [e[1] for e in dfa['edges'] if e[0]==state and e[2]==c]
    if len(next_states)==1:
        return next_states[0]
    else:
        return 'error'

def accepts(dfa,chars):
    state = dfa['start']
    for i in range(len(chars)):
        c = chars[i]
        state1 = edge(dfa,state,c)
        print('traverse edge',state,c,state1)
        if state1=='error':
            return False
        else:
            state=state1
    return state in dfa['final']
    

In [11]:
accepts(dfa1,'abac')

traverse edge 0 a 1
traverse edge 1 b 0
traverse edge 0 a 1
traverse edge 1 c 0


True

# The epsilon closure of a set of nodes

In [82]:
def epsilon_edge(NFA,S):
    ''' returns all states reachable from a state in s by an edge labelled epsilon '''
    return {e[1] for e in NFA['edges'] if e[0] in S and e[2]=='epsilon'}

def closure(NFA,S):
    ''' returns set of states reachable from a state in S following epsilon edges'''
    T=S
    next_states = epsilon_edge(NFA,S)
    #print('epsilon edges',T,'epsilon',T.union(next_states))
    while not next_states.issubset(T):
        T = T.union(next_states)
        next_states = epsilon_edge(NFA,T)
        #print('epsilon edges',T,'epsilon',T.union(next_states))
    return T
     


In [13]:
nfa1 = {'start':0,'final':{0,2},'edges':{(0,1,'a'),(1,0,'b'),(1,0,'c'),(1,2,'b'),(2,0,'c')}}

In [14]:
nfa2 = {'start':0,'final':{0,2},
        'edges':{(0,1,'epsilon'),(1,2,'epsilon'),(1,3,'epsilon'),(2,3,'epsilon'),
                 (1,3,'a'),(2,2,'a'),(3,3,'b'),(2,3,'c'),(3,2,'epsilon')}}

In [15]:
closure(nfa2,{0})

epsilon edges {0} epsilon {0, 1}
epsilon edges {0, 1} epsilon {0, 1, 2, 3}
epsilon edges {0, 1, 2, 3} epsilon {0, 1, 2, 3}


{0, 1, 2, 3}

# NFA accepting a string

In [37]:
def dfa_edge(nfa,s,c):
    ''' returns closure of all states reachable from a state in s by an edge labelled c '''
    s1 = {e[1] for e in nfa['edges'] if e[0] in s and e[2]==c}
    s2 = closure(nfa,s1)
    return s2

def is_final(nfa,state):
    ''' return True if the nfa state is a final state 
        we also allow for a single final state instead of a set 
    '''
    final_states = nfa['final']
    final_states = final_states if (type(final_states)==set) else {nfa['final']}
    return len(state.intersection(final_states))>0

def nfa_accepts(nfa,chars):
    ''' returns True if the nfa accepts the string of chars '''
    state = closure(nfa,{nfa['start']})
    for i in range(len(chars)):
        c = chars[i]
        state1=state
        state = dfa_edge(nfa,state,c)
        print('dfa_edge',state1,c,state)
    return is_final(nfa,state)

In [38]:
is_final({'final':10},{10,11})

True

In [39]:
print(nfa1)
nfa_accepts(nfa1,'abcabac')

{'start': 0, 'final': {0, 2}, 'edges': {(1, 0, 'b'), (1, 2, 'b'), (0, 1, 'a'), (1, 0, 'c'), (2, 0, 'c')}}
epsilon edges {0} epsilon {0}
epsilon edges {1} epsilon {1}
dfa_edge {0} a {1}
epsilon edges {0, 2} epsilon {0, 2}
dfa_edge {1} b {0, 2}
epsilon edges {0} epsilon {0}
dfa_edge {0, 2} c {0}
epsilon edges {1} epsilon {1}
dfa_edge {0} a {1}
epsilon edges {0, 2} epsilon {0, 2}
dfa_edge {1} b {0, 2}
epsilon edges {1} epsilon {1}
dfa_edge {0, 2} a {1}
epsilon edges {0} epsilon {0}
dfa_edge {1} c {0}


True

In [18]:
nfa_accepts(nfa2,'abaac')

epsilon edges {0} epsilon {0, 1}
epsilon edges {0, 1} epsilon {0, 1, 2, 3}
epsilon edges {0, 1, 2, 3} epsilon {0, 1, 2, 3}
epsilon edges {2, 3} epsilon {2, 3}
dfa_edge {0, 1, 2, 3} a {2, 3}
epsilon edges {3} epsilon {2, 3}
epsilon edges {2, 3} epsilon {2, 3}
dfa_edge {2, 3} b {2, 3}
epsilon edges {2} epsilon {2, 3}
epsilon edges {2, 3} epsilon {2, 3}
dfa_edge {2, 3} a {2, 3}
epsilon edges {2} epsilon {2, 3}
epsilon edges {2, 3} epsilon {2, 3}
dfa_edge {2, 3} a {2, 3}
epsilon edges {3} epsilon {2, 3}
epsilon edges {2, 3} epsilon {2, 3}
dfa_edge {2, 3} c {2, 3}


True

# Homework problem 1: longest match
Define the function longest match which returns the longest prefix of a string accepted by the NFA (or DFA)

# NFA to DFA
To convert an NFA to a DFA we do a breadth first search of the graph of NFA states
that are reachable from the start state with dfa_edge

# Homework problem 2: nfa to dfa
Write the nfa_to_dfa(nfa) function

# NFA with multiple final states
A tokenizer will have different final states for different tokens
and the final states should be listed in priority, so if a token matches 2 final state,
then the first one determines the token. 

# Homework problem 3: Tokenizer
Write a tokenizer which accepts a list of token definitions of the form
```
(token_name,  NFA)
```
and generates a DFA for recognizing those tokens!

# Regular Expressions to DFAs


In [19]:
def star(R):
    return ('star',R)
def bar(R1,R2):
    return ('bar',R1,R2)
def dot(R1,R2):
    return ('dot',R1,R2)


In [46]:
star(dot('a',bar('b','c')))

('star', ('dot', 'a', ('bar', 'b', 'c')))

In [77]:
def convert_RE_to_NFA(RE,start):
    ''' 
        The idea is to generate an NFA from a regular expression
        and to give it a starting number for the states and keep track of the highest state generated
    '''
    if type(RE)==str:
        return {'start':start,'final':start+1,'last':start+1,'edges':((start,start+1,RE),)}
    elif RE[0]=='dot':
        R1 = convert_RE_to_NFA(RE[1],start+2)
        R2 = convert_RE_to_NFA(RE[2],R1['last']+1)
        #print('dot r1',RE,'\n',R1)
        #print('dot r2',RE,'\n',R2)
        return {'start':start,'final':start+1,'last':R2['last'],
            'edges':((start,R1['start'],'epsilon'),)
                     +R1['edges'] 
                    + ((R1['final'],R2['start'],'epsilon'),)
                     +R2['edges']
                     + ((R2['final'],start+1,'epsilon'),)}

    elif RE[0]=='bar':
        R1 = convert_RE_to_NFA(RE[1],start+2)
        R2 = convert_RE_to_NFA(RE[2],R1['last']+1)
        #print('bar r1',RE,'\n',R1)
        #print('bar r2',RE,'\n',R2)
        return {'start':start,'final':start+1,'last':R2['last'],
            'edges':(     (start,R1['start'],'epsilon'),) 
                        + R1['edges']
                        + ((R1['final'],start+1,'epsilon'),(start,R2['start'],'epsilon'),) 
                        + R2['edges']+
                          ((R2['final'],start+1,'epsilon'),)  }
    elif RE[0]=='star':
        R1 = convert_RE_to_NFA(RE[1],start+2)
        #print('star r1',RE,'\n',R1)
        return {'start':start,'final':start+1,'last':R1['last'],
            'edges':((start,R1['start'],'epsilon'),
                     (R1['start'],start+1,'epsilon'))  
                    + R1['edges'] 
                    + ((R1['final'],R1['start'],'epsilon'),)}


In [78]:
NFA0 = convert_RE_to_NFA(star(dot('a',bar('b','c'))),10)
NFA0

{'start': 10,
 'final': 11,
 'last': 21,
 'edges': ((10, 12, 'epsilon'),
  (12, 11, 'epsilon'),
  (12, 14, 'epsilon'),
  (14, 15, 'a'),
  (15, 16, 'epsilon'),
  (16, 18, 'epsilon'),
  (18, 19, 'b'),
  (19, 17, 'epsilon'),
  (16, 20, 'epsilon'),
  (20, 21, 'c'),
  (21, 17, 'epsilon'),
  (17, 13, 'epsilon'),
  (13, 12, 'epsilon'))}

In [83]:
nfa_accepts(NFA0,'abacac')

dfa_edge {10, 11, 12, 14} a {16, 18, 20, 15}
dfa_edge {16, 18, 20, 15} b {11, 12, 13, 14, 17, 19}
dfa_edge {11, 12, 13, 14, 17, 19} a {16, 18, 20, 15}
dfa_edge {16, 18, 20, 15} c {11, 12, 13, 14, 17, 21}
dfa_edge {11, 12, 13, 14, 17, 21} a {16, 18, 20, 15}
dfa_edge {16, 18, 20, 15} c {11, 12, 13, 14, 17, 21}


True